In [1]:
mydir = "/share/ScratchGeneral/anncuo/OneK1K/"

In [2]:
list.files(paste0(mydir,"grm"))

[1] "grm.bed"        "grm.bim"        "grm.fam"        "grm.grm.bin"   
 [5] "grm.grm.id"     "grm.grm.N.bin"  "grm.hh"         "grm.log"       
 [9] "grm.rel.grm.id" "y_par.snps"

In [38]:
df = read.table(paste0(mydir,"grm/grm.fam"))
tail(df)

,V1,V2,V3,V4,V5,V6
,<int>,<chr>,<int>,<int>,<int>,<int>
1081,1099,848_2,0,0,2,-9
1082,1100,914_2,0,0,2,-9
1083,1101,915_2,0,0,2,-9
1084,1102,932_2,0,0,1,-9
1085,1103,926_2,0,0,2,-9
1086,1104,920_2,0,0,2,-9


In [40]:
length(unique(df$V2))

[1] 1086

In [ ]:
# goal: a donor x donor kinship matrix

In [3]:
# attempt: using function from https://gist.github.com/explodecomputer/7150052
readGRM <- function(rootname)
{
    bin.file.name <- paste(rootname, ".grm.bin", sep="")
    n.file.name <- paste(rootname, ".grm.N.bin", sep="")
    id.file.name <- paste(rootname, ".grm.id", sep="")

    cat("Reading IDs\n")
    id <- read.table(id.file.name, colClasses="character")
    n <- dim(id)[1]
    cat("Reading GRM\n")
    bin.file <- file(bin.file.name, "rb")
    grm <- readBin(bin.file, n=n*(n+1)/2, what=numeric(0), size=4)
    close(bin.file)
    cat("Reading N\n")
    n.file <- file(n.file.name, "rb")
    N <- readBin(n.file, n=n*(n+1)/2, what=numeric(0), size=4)
    close(n.file)

    cat("Creating data frame\n")
    l <- list()
    for(i in 1:n)
    {
        l[[i]] <- 1:i
    }
    col1 <- rep(1:n, 1:n)
    col2 <- unlist(l)
    grm <- data.frame(id1=col1, id2=col2, N=N, grm=grm)	

    ret <- list()
    ret$grm <- grm
    ret$id <- id
    return(ret)
}

In [4]:
rootname = "/share/ScratchGeneral/anncuo/OneK1K/grm/grm"

In [5]:
grm = readGRM(rootname)

Reading IDs
Reading GRM
Reading N
Creating data frame


In [7]:
str(grm)

List of 2
 $ grm:'data.frame':	590241 obs. of  4 variables:
  ..$ id1: int [1:590241] 1 2 2 3 3 3 4 4 4 4 ...
  ..$ id2: int [1:590241] 1 1 2 1 2 3 1 2 3 4 ...
  ..$ N  : num [1:590241] 497039 496776 497965 496791 497710 ...
  ..$ grm: num [1:590241] 0.995624 -0.001683 0.990942 -0.000266 0.00444 ...
 $ id :'data.frame':	1086 obs. of  2 variables:
  ..$ V1: chr [1:1086] "1" "2" "3" "4" ...
  ..$ V2: chr [1:1086] "1" "2" "3" "4" ...


In [9]:
dim(grm$grm)

[1] 590241      4

In [23]:
x = grm$grm

In [36]:
head(x)

,id1,id2,grm
,<int>,<int>,<dbl>
1,1,1,0.9956240654
2,2,1,-0.0016830340
3,2,2,0.9909417629
4,3,1,-0.0002661081
5,3,2,0.0044398019
6,3,3,0.9953926206


In [43]:
ids = grm$id
tail(ids)

,V1,V2
,<chr>,<chr>
1081,1099,848_2
1082,1100,914_2
1083,1101,915_2
1084,1102,932_2
1085,1103,926_2
1086,1104,920_2


In [44]:
colnames(ids) = c("FAMID","INDID")

In [45]:
write.csv(ids,"/share/ScratchGeneral/anncuo/OneK1K/input_files_CellRegMap/grm_ids.csv")

In [25]:
write.csv(x,"/share/ScratchGeneral/anncuo/OneK1K/input_files_CellRegMap/grm_long.csv")

In [24]:
length(unique(x$id1))
length(unique(x$id2))

[1] 1086

[1] 1086